## Imports

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import holidays
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import lightgbm as lgb

C:\Users\catar\AppData\Local\Temp\ipykernel_9584\2839718103.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Read data

### Functions

In [ ]:
# def prepare_data(data_folder) OLD:
#     """Loads & preprocesses data from given test folder."""
    
#     # 1. Read data
#     df_prod_sales = pd.read_csv(f"{data_folder}/product_structures_sales.csv")
#     df_prices = pd.read_csv(f"{data_folder}/product_prices_leaflets.csv")
#     df_chains = pd.read_csv(f"{data_folder}/chain_campaigns.csv")

#     # TESTEEE!!! ignore columns pvp_was, discount, flag_promo, leaflet (é esperado q eles removam ao enviar os dados) -> mas isto seria só para os competitors
#     df_prices.loc[df_prices['competitor'].isin(['competitorA', 'competitorB']), 'discount']=None
#     df_prices.loc[df_prices['competitor'].isin(['competitorA', 'competitorB']), 'pvp_was']=None
#     df_prices.loc[df_prices['competitor'].isin(['competitorA', 'competitorB']), 'flag_promo']=None
#     df_prices.loc[df_prices['competitor'].isin(['competitorA', 'competitorB']), 'leaflet']=None

#     # 2. Convert time_key to datetime
#     df_prod_sales["date"] = pd.to_datetime(df_prod_sales["time_key"].astype(str), format="%Y%m%d")
#     df_prices["date"] = pd.to_datetime(df_prices["time_key"].astype(str), format="%Y%m%d")
#     df_chains["start_date"] = pd.to_datetime(df_chains["start_date"])
#     df_chains["end_date"] = pd.to_datetime(df_chains["end_date"])
#     df_prod_sales['structure_level_1'] = df_prod_sales['structure_level_1'].astype("str")
#     df_prod_sales['structure_level_2'] = df_prod_sales['structure_level_2'].astype("str")
#     df_prod_sales['structure_level_3'] = df_prod_sales['structure_level_3'].astype("str")
#     df_prod_sales['structure_level_4'] = df_prod_sales['structure_level_4'].astype("str")

#     # 3. Explode chain_campaigns by date
#     df_chains["date_range"] = df_chains.apply(lambda row: pd.date_range(start=row["start_date"], end=row["end_date"]), axis=1)
#     df_chains = df_chains.explode("date_range").rename(columns={"date_range": "date"}).drop(["start_date", "end_date"], axis=1)

#     # 4. Merge datasets
#     df = df_prices.merge(df_prod_sales.drop("time_key", axis=1), on=["sku", "date"], how="left").merge(df_chains, on=["competitor", "date"], how="left")

#     # 5. Remove rows where quantity is missing
#     df = df.dropna(subset=["quantity"]).drop("time_key", axis=1)

#     # 6. Add Holidays & Weather Data
#     df_holidays = pd.DataFrame(holidays.Portugal(years=range(df["date"].min().year, df["date"].max().year + 1), subdiv="11").items(), columns=["date", "holiday"])
#     df_holidays["date"] = pd.to_datetime(df_holidays["date"])
#     df_holidays["holiday_importance"] = df_holidays["holiday"].str.lower().str.contains("christmas|new year|easter", regex=True).astype(int) + 1

#     try:
#         url = "https://archive-api.open-meteo.com/v1/archive"
#         params = {
#             "latitude": 39.6945,
#             "longitude": -8.1306,
#             "start_date": df["date"].min().strftime("%Y-%m-%d"),
#             "end_date": df["date"].max().strftime("%Y-%m-%d"),
#             "daily": ["apparent_temperature_mean", "precipitation_sum"],
#             "timezone": "Europe/London"
#         }
#         r = requests.get(url, params=params)
#         r.raise_for_status() 
#         jsondata = r.json()
#         daily_data = jsondata['daily']
#         df_weather = (pd.DataFrame.from_dict(daily_data))
#         ## Remove decimal cases
#         df_weather['date'] = pd.to_datetime(df_weather['time'])
#         df_weather['apparent_temperature_mean'] = df_weather['apparent_temperature_mean'].astype("int")
#         df_weather['precipitation_sum'] = df_weather['precipitation_sum'].astype("int")
#     except requests.exceptions.HTTPError as errh: 
#         print("HTTP Error while trying to extract weather info ")
#         print(errh.args[0])
#         raise
#     except requests.exceptions.ReadTimeout as errrt: 
#         print("Time out while trying to extract weather info")
#         print(errrt.args[0])
#         raise
#     except requests.exceptions.ConnectionError as conerr: 
#         print("Connection error while trying to extract weather info ") 
#         print(conerr.args[0])
#         raise

#     # 7. Add new features
#     days_per_group = df.groupby('structure_level_1')['date'].nunique().reset_index()
#     days_per_group.columns = ['structure_level_1', 'n_days']

#     demand_stats = (
#         df.groupby(['structure_level_1', 'sku'])
#         .agg(count_days=('date', 'nunique'),
#             quantity=('quantity', 'sum'))
#         .reset_index()
#     )
#     demand_stats = demand_stats.merge(days_per_group, on='structure_level_1')
#     demand_stats['average_demand_interval'] = demand_stats['n_days'] / demand_stats['count_days']

#     ## 7.1 Product classes
#     # ABC classification
#     demand_stats['rank'] = demand_stats.groupby('structure_level_1')['quantity'] \
#                                     .rank(method='first', ascending=False)
#     demand_stats = demand_stats.sort_values(['structure_level_1', 'rank'])
#     demand_stats['total_quantity'] = demand_stats.groupby('structure_level_1')['quantity'].transform('sum')
#     demand_stats['cumulative_quantity'] = demand_stats.groupby('structure_level_1')['quantity'].cumsum()
#     demand_stats['cumulative_pct'] = demand_stats['cumulative_quantity'] / demand_stats['total_quantity']
#     demand_stats['abc'] = np.select(
#         [demand_stats['cumulative_pct'] <= 0.80,
#         demand_stats['cumulative_pct'] <= 0.95],
#         ['A', 'B'],
#         default='C')

#     ## XYZ classification
#     unique_dates = df['date'].drop_duplicates()
#     unique_sku_pairs = df[['structure_level_1', 'sku']].drop_duplicates()
#     sku_date_combinations = unique_sku_pairs.merge(unique_dates.to_frame(), how='cross')

#     df_slim = df[['structure_level_1', 'sku', 'date', 'quantity']]
#     df_full = sku_date_combinations.merge(df_slim, on=['structure_level_1', 'sku', 'date'], how='left')
#     df_full['quantity'] = df_full['quantity'].fillna(0)

#     sales_stats = (
#         df_full.groupby(['structure_level_1', 'sku'])
#         .agg(std_sales=('quantity', 'std'),
#             avg_sales=('quantity', 'mean'))
#         .reset_index())

#     sales_stats = sales_stats.merge(
#         demand_stats[['structure_level_1', 'sku', 'average_demand_interval', 'abc']],
#         on=['structure_level_1', 'sku'],
#         how='left')

#     sales_stats['cv'] = sales_stats['std_sales'] / sales_stats['avg_sales']
#     sales_stats['cv_2'] = sales_stats['cv'] ** 2
#     sales_stats['xyz'] = np.select(
#         [sales_stats['cv'] <= 0.5, sales_stats['cv'] <= 1],
#         ['X', 'Y'],
#         default='Z'
#     )

#     # SEIL classification
#     sales_stats['seil'] = np.select(
#         [
#             (sales_stats['average_demand_interval'] < 1.32) & (sales_stats['cv_2'] < 0.49),
#             (sales_stats['average_demand_interval'] < 1.32) & (sales_stats['cv_2'] >= 0.49),
#             (sales_stats['average_demand_interval'] >= 1.32) & (sales_stats['cv_2'] < 0.49)
#         ],
#         ['smooth', 'erratic', 'intermittent'],
#         default='lumpy'
#     )

#     df_class = sales_stats[['structure_level_1', 'sku', 'abc', 'seil', 'xyz']]

#     ## 7.2 Calendar Features
#     df['month'] = df['date'].dt.month
#     df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
#     df['week_of_month'] = df['date'].apply(lambda d: (d.day - 1) // 7 + 1)

#     ## 7.3 Promo Participation and Avg discount (Structure Level 3 x competitor)
#     df_avg_discount = df.copy()
#     df_avg_discount['discount_qty_pf1'] = (df_avg_discount['discount']*df_avg_discount['quantity']*df_avg_discount['flag_promo'])
#     df_avg_discount['qty_pf1'] = (df_avg_discount['quantity']*df_avg_discount['flag_promo'])
#     df_avg_discount = df_avg_discount.loc[((df_avg_discount['flag_promo']==1))]

#     df_avg_discount = (df_avg_discount.groupby(['structure_level_3', 'competitor'])
#                                             .agg({'discount_qty_pf1': "sum",
#                                                     'qty_pf1': "sum",
#                                                     'discount': "mean"}))
#     df_avg_discount['avg_discount_w_L3'] = df_avg_discount['discount_qty_pf1']/df_avg_discount['qty_pf1']
#     df_avg_discount = df_avg_discount.rename(columns={'discount':'avg_discount'})[['avg_discount_w_L3']]
#     df_avg_discount = df_avg_discount.reset_index()
#     df_avg_discount['avg_discount_w_L3'] = df_avg_discount['avg_discount_w_L3'].round(2)

#     df_pp = df.copy()
#     df_pp['qty_pf1'] = (df_pp['quantity']*df_pp['flag_promo'])
#     df_pp = (df_pp.groupby(['structure_level_3', 'competitor'])
#                 .agg(qty_pf1 = ("qty_pf1","sum"),
#                     quantity = ("quantity","sum"),
#                     count_pf1 = ("flag_promo","sum"),
#                     count_all = ("flag_promo","count")))
#     df_pp['promo_part_w_L3'] = df_pp['qty_pf1']/df_pp['quantity']
#     df_pp['promo_part'] = df_pp['count_pf1']/df_pp['count_all']
#     df_pp = df_pp[['promo_part_w_L3']]
#     df_pp = df_pp.reset_index()
#     df_pp['promo_part_w_L3'] = df_pp['promo_part_w_L3'].round(2)

#     df_new = (df.merge(df_holidays, on=['date'], how='left')
#                           .merge(df_weather.drop('time', axis=1), on=['date'], how='left')
#                           .merge(df_class, on=['structure_level_1','sku'], how='left')
#                           .merge(df_avg_discount, on=['structure_level_3','competitor'], how='left')
#                           .merge(df_pp, on=['structure_level_3','competitor'], how='left'))
#     ## fill with nan non discount structL3
#     df_new[['avg_discount_w_L3','promo_part_w_L3']] = df_new[['avg_discount_w_L3','promo_part_w_L3']].fillna(value=0)

#     ## limit discount values (fix errors)
#     df_new['discount'] = df_new['discount'].clip(lower=0)

#     ## fill nan values
#     df_new[['holiday']] = df_new[['holiday']].fillna(value="")
#     df_new[['leaflet']] = df_new[['leaflet']].fillna(value="")
#     df_new[['chain_campaign']] = df_new[['chain_campaign']].fillna(value="")
#     df_new[['holiday_importance']] = df_new[['holiday_importance']].fillna(value=0)

#     ## fix data types
#     df_new['holiday_importance'] = df_new['holiday_importance'].astype("int")

#     ## 8. Convert data to modelling structure
#     df_chain = df_new.loc[df_new.competitor=='chain']
#     df_chain = df_chain.rename(columns={"discount": "discount_chain","leaflet": "leaflet_chain"})

#     df_model = df_new.loc[df_new.competitor!='chain']

#     df_model = (df_model.merge(df_chain[['sku','date','discount_chain','leaflet_chain']], on=['sku','date'], how='left'))

#     ### TESTE IGNORE COLUMNS discount flag_promo leaflet pvp_was
#     df_model = df_model.drop(['discount', 'flag_promo', 'leaflet', 'pvp_was'], axis=1)
#     return df_model

In [2]:
def prepare_data(sku_input, date_input):
    """Loads & preprocesses data from given test folder."""
    
    # 1. Read history data
    df_trained = pd.read_csv("prepared_data/data.csv")
    df_trained['date'] = pd.to_datetime(df_trained['date'])
    max_date_history = pd.to_datetime(df_trained['date'].max())

    # 2. Get predictions range dates
    date_pred_range = list(pd.date_range(start=max_date_history+ pd.Timedelta(days=1), end=date_input, freq='D').strftime('%Y-%m-%d'))
    df_date_pred_range = pd.DataFrame(date_pred_range, columns=['date'])
    df_date_pred_range['date'] = pd.to_datetime(df_date_pred_range['date'])

    # 3. Prepare test data set
    ## 3.1 Features from history data
    df_test = (df_trained.loc[df_trained.sku==sku_input][['sku', 'competitor', 'structure_level_1','abc', 'xyz', 'seil', 'structure_level_3',
                                                        'promo_part_w_sku', 'avg_discount_w_sku', 'avg_discount_w_L3', 'promo_part_w_L3']]
                                                        .drop_duplicates())
    ## 3.2 Lag_7 features (if date_pred_range <7, else we need to fill them with recursive forecast)
    df_test = df_test.merge(df_date_pred_range, how='cross')
    df_test["date_lag_7"] = df_test["date"] - pd.to_timedelta(7, unit="D")
    ### lag1 usefull in the model recursive forecast 
    df_test["date_lag_1"] = df_test["date"] - pd.to_timedelta(1, unit="D")

    ## 3.3 Days since last promo feature (for the 1st day to predict, the other dates need to be derive from recursive forecast)
    df_last_promo_dates = (df_trained[df_trained["flag_promo"] == 1]
                        .groupby(["sku","competitor"])["date"].max()
                        .reset_index()
                        .rename(columns={"date": "last_promo_date"}))

    df_test = (df_test.merge((df_trained[['sku', 'date', 'competitor', 'pvp_is', 'discount']]
                                .rename(columns={'date':'date_lag_7',
                                                'pvp_is':'pvp_is_lag_7',
                                                'discount':'discount_lag_7'})), on=['sku', 'date_lag_7','competitor'], how='left')
                    .merge(df_last_promo_dates, on=['sku', 'competitor'], how='left'))
    df_test["days_since_last_promo"] = (df_test["date"] - df_test["last_promo_date"]).dt.days
    df_test["days_since_last_promo"] = df_test["days_since_last_promo"].clip(lower=0)

    # 4. Calendar Features
    df_test['month'] = df_test['date'].dt.month
    df_test['day_of_week'] = df_test['date'].dt.dayofweek  # Monday=0, Sunday=6
    df_test['week_of_month'] = df_test['date'].apply(lambda d: (d.day - 1) // 7 + 1)


    # 5. Add Holidays & Weather Data
    df_holidays = pd.DataFrame(holidays.Portugal(years=range(df_test["date"].min().year, df_test["date"].max().year + 1), subdiv="11").items(), columns=["date", "holiday"])
    df_holidays["date"] = pd.to_datetime(df_holidays["date"])
    df_holidays["holiday_importance"] = df_holidays["holiday"].str.lower().str.contains("christmas|new year|easter", regex=True).astype(int) + 1
    df_holidays = df_holidays[['date', 'holiday_importance']]

    try:
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": 39.6945,
            "longitude": -8.1306,
            "start_date": df_test["date"].min().strftime("%Y-%m-%d"),
            "end_date": df_test["date"].max().strftime("%Y-%m-%d"),
            "daily": ["apparent_temperature_mean", "precipitation_sum"],
            "timezone": "Europe/London"
        }
        r = requests.get(url, params=params)
        r.raise_for_status() 
        jsondata = r.json()
        daily_data = jsondata['daily']
        df_weather = (pd.DataFrame.from_dict(daily_data))
        ## Remove decimal cases
        df_weather['date'] = pd.to_datetime(df_weather['time'])
        df_weather['apparent_temperature_mean'] = df_weather['apparent_temperature_mean'].astype("int")
        df_weather['precipitation_sum'] = df_weather['precipitation_sum'].astype("int")
    except requests.exceptions.HTTPError as errh: 
        print("HTTP Error while trying to extract weather info ")
        print(errh.args[0])
        raise
    except requests.exceptions.ReadTimeout as errrt: 
        print("Time out while trying to extract weather info")
        print(errrt.args[0])
        raise
    except requests.exceptions.ConnectionError as conerr: 
        print("Connection error while trying to extract weather info ") 
        print(conerr.args[0])
        raise

    # 6. Add last pvp_was from history
    df_trained = df_trained.sort_values(by=['date'])
    df_last_pvp_was = df_trained.groupby(['sku', 'competitor']).tail(1).reset_index(drop=True)[['sku','competitor', 'pvp_was']].rename(columns={"pvp_was":"last_pvp_was_train"})

    df_test = (df_test.merge(df_holidays, on=['date'], how='left')
                      .merge(df_weather.drop('time', axis=1), on=['date'], how='left')
                      .merge(df_last_pvp_was,on=['sku', 'competitor'], how='left'))
    df_test[['holiday_importance']] = df_test[['holiday_importance']].fillna(value=0)
    df_test['structure_level_1'] = df_test['structure_level_1'].astype("str")
    df_test['structure_level_3'] = df_test['structure_level_3'].astype("str")
    return df_test

In [3]:
def load_model(sku_input, df_test):
    """Loads model for the input sku"""

    TMP_DIR = "models"
    # 1. Get structure_level_1 for given sku
    struct = df_test.loc[df_test.sku==sku_input]['structure_level_1'].head()[0]

    # 2️. Build model filename based on structure_level_1 & competitor
    model_compA_filename = f"final_model_it_structL1_{struct}_competitorA.pickle"
    model_compA_path = os.path.join(TMP_DIR, model_compA_filename)
    model_compB_filename = f"final_model_it_structL1_{struct}_competitorB.pickle"
    model_compB_path = os.path.join(TMP_DIR, model_compB_filename)

    # 3️. Load model if file exists, otherwise raise error
    if os.path.exists(model_compA_path):
        print(f"Loading model: {model_compA_filename}")
        model_compA = joblib.load(model_compA_path)
    else:
        raise FileNotFoundError(f"Model file '{model_compA_filename}' not found in '{TMP_DIR}'. Check model existence.")

    if os.path.exists(model_compB_path):
        print(f"Loading model: {model_compB_filename}")
        model_compB = joblib.load(model_compB_path)
    else:
        raise FileNotFoundError(f"Model file '{model_compB_filename}' not found in '{TMP_DIR}'. Check model existence.")
    
    return model_compA, model_compB

In [ ]:
# def get_predictions(model_compA, model_compB, df_test, sku_input, date_input):
#     """ Generate predictions for provided models, sku and date """


#     df_compA = df_test.loc[((df_test.sku==sku_input) & (df_test.date==date_input) & (df_test.competitor=='competitorA'))].drop(columns=excluded_cols)
#     df_compB = df_test.loc[((df_test.sku==sku_input) & (df_test.date==date_input) & (df_test.competitor=='competitorB'))].drop(columns=excluded_cols)

#     pvp_is_competitorA = float(model_compA.predict(df_compA)[0])
#     pvp_is_competitorB = float(model_compB.predict(df_compB)[0])

#     return pvp_is_competitorA, pvp_is_competitorB

In [ ]:
def get_predictions(model_compA, model_compB, df_test, sku_input, date_input):
    """ Generate predictions for provided models, sku and date """

    df_test['discount_pred_competitorA'] = None
    df_test['discount_pred_competitorB'] = None
    df_test['pvp_is_competitorA'] = None
    df_test['pvp_is_competitorB'] = None
    for d in df_test['date'].drop_duplicates():
        print("\n\n******", d)
        df_test = df_test.sort_values(["sku", "competitor", "date"])

        ## 4.1 Check if lag7 features need to be filled
        if len(df_test.loc[((df_test.sku==sku_input) & (df_test.date==d) & (df_test.discount_lag_7.isnull()))])>0:
            print("\nNeed to fill lag7 features for date: ", d)
            df_test = (df_test.merge(df_test.rename(columns={'date':'date_lag_7',
                                                    'pvp_is':'pvp_is_lag_7',
                                                    'discount':'discount_lag_7'}), ['sku', 'competitor'], 'left'))
        ## 4.2 Get predictions
        df_compA = df_test.loc[((df_test.sku==sku_input) & (df_test.date==d) & (df_test.competitor=='competitorA'))]
        df_compB = df_test.loc[((df_test.sku==sku_input) & (df_test.date==d) & (df_test.competitor=='competitorB'))]

        discount_pred_competitorA = float(model_compA.predict(df_compA)[0])
        discount_pred_competitorB = float(model_compB.predict(df_compB)[0])

        print("discount_pred_competitorA: ", discount_pred_competitorA)
        print("discount_pred_competitorB: ", discount_pred_competitorB)

        df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 'discount_pred_competitorA'] = discount_pred_competitorA
        df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 'discount_pred_competitorB'] = discount_pred_competitorB

        ## 4.3 Convert from discount to price prediction
        df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 
                    'pvp_is_competitorA'] = (df_test['last_pvp_was_train']*(1-df_test['discount_pred_competitorA']))
        df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 
                    'pvp_is_competitorB'] = (df_test['last_pvp_was_train']*(1-df_test['discount_pred_competitorB']))

        ## 4.4 Update days_since_last_promo (if predicted discount >0 and if there are more dates to predict)
        if len(df_test.loc[df_test.date>d])>0:
            if round(discount_pred_competitorA, 2)>0:
                print("\n update days_since_last_promo for compA")
                df_test.loc[((df_test.sku==sku_input) & (df_test.competitor=='competitorA')), 
                            'days_since_last_promo'] = (df_test.groupby(["sku", "competitor"])["date"]
                                                               .transform(lambda dates: (dates > d).cumsum()*(dates > d)))

            if round(discount_pred_competitorB, 2)>0:
                print("\n update days_since_last_promo for compB")
                df_test.loc[((df_test.sku==sku_input) & (df_test.competitor=='competitorB')), 
                            'days_since_last_promo'] = (df_test.groupby(["sku", "competitor"])["date"]
                                                               .transform(lambda dates: (dates > d).cumsum()*(dates > d)))
                
    pvp_is_competitorA = round(float(df_test.loc[((df_test.sku==sku_input) & 
                                            (df_test.date==date_input) & 
                                            (df_test.competitor=='competitorA'))].pvp_is_competitorA.iloc[0]),2)
    pvp_is_competitorB = round(float(df_test.loc[((df_test.sku==sku_input) & 
                                            (df_test.date==date_input) & 
                                            (df_test.competitor=='competitorB'))].pvp_is_competitorB.iloc[0]),2)
    
    return pvp_is_competitorA,pvp_is_competitorB

### Code

In [ ]:
## OLD

# ## Load input

# sku_input =3578
# time_key_input = 20241101
# date_input = pd.to_datetime(str(time_key_input), format='%Y%m%d').strftime('%Y-%m-%d')
# df_test = prepare_data("test")
# df_test_input = df_test.loc[((df_test.sku==sku_input) & (df_test.date==date_input))]



# model_compA, model_compB = load_model(sku_input, df_test)

# pvp_is_competitorA, pvp_is_competitorB = get_predictions(model_compA, model_compB, sku_input, date_input)

In [21]:
## 1. Load input
sku_input =3578
time_key_input = 20241101
date_input = pd.to_datetime(str(time_key_input), format='%Y%m%d')

## 2. Prepare test dataset
df_test = prepare_data(sku_input, date_input)

## 3. Load model
model_compA, model_compB = load_model(sku_input, df_test)

## 4. Get predictions
pvp_is_competitorA, pvp_is_competitorB = get_predictions(model_compA, model_compB, df_test, sku_input, date_input)

print("\npvp_is_competitorA: ", pvp_is_competitorA)
print("\npvp_is_competitorB: ", pvp_is_competitorB)


Loading model: final_model_it_structL1_1_competitorA.pickle
Loading model: final_model_it_structL1_1_competitorB.pickle


****** 2024-10-29 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485


****** 2024-10-30 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485


****** 2024-10-31 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485


****** 2024-11-01 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485

pvp_is_competitorA:  62.30414663240917

pvp_is_competitorB:  62.30077832704998


In [17]:
float(pvp_is_competitorA.pvp_is_competitorA)

C:\Users\catar\AppData\Local\Temp\ipykernel_9584\3079560328.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(pvp_is_competitorA.pvp_is_competitorA)


62.30414663240917

In [8]:
df_test

,sku,competitor,structure_level_1,abc,xyz,seil,structure_level_3,promo_part_w_sku,avg_discount_w_sku,avg_discount_w_L3,...,day_of_week,week_of_month,holiday_importance,apparent_temperature_mean,precipitation_sum,last_pvp_was_train,discount_pred_competitorA,discount_pred_competitorB,pvp_is_competitorA,pvp_is_competitorB
0,3578,competitorA,1,A,X,smooth,10207,0.021630,0.208339,0.21923,...,1,5,0.0,12,0,62.31,None,None,None,None
1,3578,competitorA,1,A,X,smooth,10207,0.021630,0.208339,0.21923,...,2,5,0.0,9,6,62.31,None,None,None,None
2,3578,competitorA,1,A,X,smooth,10207,0.021630,0.208339,0.21923,...,3,5,0.0,9,6,62.31,None,None,None,None
3,3578,competitorA,1,A,X,smooth,10207,0.021630,0.208339,0.21923,...,4,1,1.0,10,13,62.31,None,None,None,None
4,3578,competitorB,1,A,X,smooth,10207,0.183845,0.210254,0.20780,...,1,5,0.0,12,0,62.31,None,None,None,None
5,3578,competitorB,1,A,X,smooth,10207,0.183845,0.210254,0.20780,...,2,5,0.0,9,6,62.31,None,None,None,None
6,3578,competitorB,1,A,X,smooth,10207,0.183845,0.210254,0.20780,...,3,5,0.0,9,6,62.31,None,None,None,None
7,3578,competitorB,1,A,X,smooth,10207,0.183845,0.210254,0.20780,...,4,1,1.0,10,13,62.31,None,None,None,None


In [92]:
df_test[['sku', 'date', 'competitor', 'days_since_last_promo']]

,sku,date,competitor,days_since_last_promo
0,3578,2024-10-29,competitorA,330
1,3578,2024-10-30,competitorA,331
2,3578,2024-10-31,competitorA,332
3,3578,2024-11-01,competitorA,333
4,3578,2024-10-29,competitorB,1
5,3578,2024-10-30,competitorB,2
6,3578,2024-10-31,competitorB,3
7,3578,2024-11-01,competitorB,4


In [ ]:
## 4. Generate predictions
df_test['discount_pred_competitorA'] = None
df_test['discount_pred_competitorB'] = None
df_test['pvp_is_competitorA'] = None
df_test['pvp_is_competitorB'] = None
for d in df_test['date'].drop_duplicates():
    print("\n\n******", d)
    df_test = df_test.sort_values(["sku", "competitor", "date"])

    ## 4.1 Check if lag7 features need to be filled
    if len(df_test.loc[((df_test.sku==sku_input) & (df_test.date==d) & (df_test.discount_lag_7.isnull()))])>0:
        print("\nNeed to fill lag7 features for date: ", d)
        df_test = (df_test.merge(df_test.rename(columns={'date':'date_lag_7',
                                                'pvp_is':'pvp_is_lag_7',
                                                'discount':'discount_lag_7'}), ['sku', 'competitor'], 'left'))
    ## 4.2 Get predictions
    df_compA = df_test.loc[((df_test.sku==sku_input) & (df_test.date==d) & (df_test.competitor=='competitorA'))]
    df_compB = df_test.loc[((df_test.sku==sku_input) & (df_test.date==d) & (df_test.competitor=='competitorB'))]

    discount_pred_competitorA = float(model_compA.predict(df_compA)[0])
    discount_pred_competitorB = float(model_compB.predict(df_compB)[0])

    print("discount_pred_competitorA: ", discount_pred_competitorA)
    print("discount_pred_competitorB: ", discount_pred_competitorB)

    df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 'discount_pred_competitorA'] = discount_pred_competitorA
    df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 'discount_pred_competitorB'] = discount_pred_competitorB

    ## 4.3 Convert from discount to price prediction
    df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 'pvp_is_competitorA'] = df_test['last_pvp_was_train']*(1-df_test['discount_pred_competitorA'])
    df_test.loc[((df_test.sku==sku_input) & (df_test.date==d)), 'pvp_is_competitorB'] = df_test['last_pvp_was_train']*(1-df_test['discount_pred_competitorB'])

    ## 4.4 Update days_since_last_promo (if predicted discount >0 and if there are more dates to predict)
    if len(df_test.loc[df_test.date>d])>0:
        if round(discount_pred_competitorA, 2)>0:
            print("\n update days_since_last_promo for compA")
            df_test.loc[((df_test.sku==sku_input) & (df_test.competitor=='competitorA')), 'days_since_last_promo'] = (df_test.groupby(["sku", "competitor"])["date"]
                                                                                                                 .transform(lambda dates: (dates > d).cumsum()*(dates > d)))

        if round(discount_pred_competitorB, 2)>0:
            print("\n update days_since_last_promo for compB")
            df_test.loc[((df_test.sku==sku_input) & (df_test.competitor=='competitorB')), 'days_since_last_promo'] = (df_test.groupby(["sku", "competitor"])["date"]
                                                                                                                 .transform(lambda dates: (dates > d).cumsum()*(dates > d)))



****** 2024-10-29 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485


****** 2024-10-30 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485


****** 2024-10-31 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485


****** 2024-11-01 00:00:00
discount_pred_competitorA:  9.393945740377387e-05
discount_pred_competitorB:  0.00014799667709865485


In [97]:
df_test[['sku', 'date', 'competitor', 'days_since_last_promo', 'discount_pred_competitorA', 'discount_pred_competitorB', 'pvp_is_competitorA', 'pvp_is_competitorB']]

,sku,date,competitor,days_since_last_promo,discount_pred_competitorA,discount_pred_competitorB,pvp_is_competitorA,pvp_is_competitorB
0,3578,2024-10-29,competitorA,330,0.000094,0.000148,62.304147,62.300778
1,3578,2024-10-30,competitorA,331,0.000094,0.000148,62.304147,62.300778
2,3578,2024-10-31,competitorA,332,0.000094,0.000148,62.304147,62.300778
3,3578,2024-11-01,competitorA,333,0.000094,0.000148,62.304147,62.300778
4,3578,2024-10-29,competitorB,1,0.000094,0.000148,62.304147,62.300778
5,3578,2024-10-30,competitorB,2,0.000094,0.000148,62.304147,62.300778
6,3578,2024-10-31,competitorB,3,0.000094,0.000148,62.304147,62.300778
7,3578,2024-11-01,competitorB,4,0.000094,0.000148,62.304147,62.300778


In [34]:
df_test

,sku,competitor,abc,xyz,seil,structure_level_3,promo_part_w_sku,avg_discount_w_sku,avg_discount_w_L3,promo_part_w_L3,date,date_lag_7,pvp_is_lag_7,discount_lag_7
0,3578,competitorA,A,X,smooth,10207,0.021630,0.208339,0.21923,0.064422,2024-10-28,2024-10-21,62.310000,0.0000
1,3578,competitorA,A,X,smooth,10207,0.021630,0.208339,0.21923,0.064422,2024-10-29,2024-10-22,62.310000,0.0000
2,3578,competitorA,A,X,smooth,10207,0.021630,0.208339,0.21923,0.064422,2024-10-30,2024-10-23,62.310000,0.0000
3,3578,competitorA,A,X,smooth,10207,0.021630,0.208339,0.21923,0.064422,2024-10-31,2024-10-24,62.310000,0.0000
4,3578,competitorA,A,X,smooth,10207,0.021630,0.208339,0.21923,0.064422,2024-11-01,2024-10-25,62.310000,0.0000
5,3578,competitorB,A,X,smooth,10207,0.183845,0.210254,0.20780,0.091532,2024-10-28,2024-10-21,49.467909,0.2061
6,3578,competitorB,A,X,smooth,10207,0.183845,0.210254,0.20780,0.091532,2024-10-29,2024-10-22,49.467909,0.2061
7,3578,competitorB,A,X,smooth,10207,0.183845,0.210254,0.20780,0.091532,2024-10-30,2024-10-23,49.467909,0.2061
8,3578,competitorB,A,X,smooth,10207,0.183845,0.210254,0.20780,0.091532,2024-10-31,2024-10-24,46.688883,0.2507
9,3578,competitorB,A,X,smooth,10207,0.183845,0.210254,0.20780,0.091532,2024-11-01,2024-10-25,46.688883,0.2507


In [23]:
df_date_pred_range

,date
0,2024-10-28
1,2024-10-29
2,2024-10-30
3,2024-10-31
4,2024-11-01


In [13]:
list(date_range_array)

['2024-10-28', '2024-10-29', '2024-10-30', '2024-10-31', '2024-11-01']

In [8]:
date_input

Timestamp('2024-11-01 00:00:00')

In [9]:
max_date_history

Timestamp('2024-10-28 00:00:00')

In [61]:
df_test_input

,sku,competitor,date,structure_level_4,structure_level_3,structure_level_2,structure_level_1,quantity,chain_campaign,month,...,holiday_importance,apparent_temperature_mean,precipitation_sum,abc,seil,xyz,avg_discount_w_L3,promo_part_w_L3,discount_chain,leaflet_chain
44388,3578,competitorB,2024-11-01,1020701,10207,102,1,5042.1888,,11,...,1,10,13,A,smooth,X,0.0,0.0,0.0,
51055,3578,competitorA,2024-11-01,1020701,10207,102,1,5042.1888,,11,...,1,10,13,A,smooth,X,0.0,0.0,0.0,


In [85]:
pvp_is_competitorA

34.67974413446305

In [84]:
pvp_is_competitorB

34.693755651405276

In [74]:
df_prices_aux_preds = pd.read_csv(f"test/product_prices_leaflets.csv")

In [77]:
df_prices_aux_preds.loc[((df_prices_aux_preds.sku==sku_input) & (df_prices_aux_preds.time_key==time_key_input))][['sku', 'time_key', 'competitor', 'pvp_was', 'discount']]

,sku,time_key,competitor,pvp_was,discount
70046,3578,20241101,chain,62.31,0.0
100958,3578,20241101,competitorB,62.31,0.0
116084,3578,20241101,competitorA,62.31,0.0
